# Case study: Shear Wave Elastography (SWE)

## A (very) brief introduction to SWE

Ultrasound shear wave elastography (SWE) has emerged as a promising technique that enables the quantitative estimation of soft tissue stiffness.  
The SWE technique relies on remotely inducing tissue displacement using the acoustic radiation force (ARF), as first proposed by Sarvazyan et al. [Sarvazyan1998].   
The ARF is generated by an ultrasound beam (also called a push beam or push pulse) and is defined in following equation:  

$$
\vec{F} = \frac{2\alpha{}\vec{I}}{c}
$$

where
* $\vec{F}$ - acoustic radiation force,
* $\alpha$ - absorption coefficient, 
* $\vec{I}$ - acoustic intensity, 
* $c$ - speed of sound in the medium. It is usually assumed that in soft tissue $c = 1540 [m/s]$.



--------------------------------------------------------------------------------

## Let's implement above estimators in GPU kernel, use it, and show some SWE images.   

### 'import' section

In [1]:
import cupy as cp
import cupyx.scipy.ndimage
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from IPython.display import HTML
# from gpu_short_course.flow_imaging import (
#     dB,
#     filter_wall_clutter_gpu,
#     iq2bmode_gpu,
#     show_flow,    
#     show_flow_cineloop,
# )

GPU:0: b'GeForce GTX TITAN X'


In [2]:
# Cfg
plt.rcParams["animation.html"] = "jshtml"
plt.ioff()

### Kernel definition
KERNEL DESCRIPTION HERE  

The kernel has the following parameters:


In [3]:
cuda_source = open("doppler.cc").read()
doppler = cp.RawKernel(cuda_source, 'doppler')

### Data loading

First, download example data.

! wget -nc -nv LINK_TO_FILE_HERE -P ./data  

-----------------

DATA DESCRIPTION HERE 

### ELASTICITY estimation using the kernel

In [ ]:
CALCULATIONS HERE

### Presentation of results 

In [ ]:
RESULTS HERE

-----------------------------------------

## References

[Sarvazyan1998] Sarvazyan, A.P.; Rudenko, O.V.; Swanson, S.D.; Fowlkes, J.B.; Emalianov, S.Y. Shear Wave Elasticity Imaging: A New Ultrasonic Technology of Medical Diagnostics. Ultrasound Med. Biol. 1998, 24, 1419–1435.
